In [1]:
# Libraries needed for basic web-scraping
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from IPython.display import IFrame
import re
import urllib # package required to interact with live webpage
import pandas as pd # will use to store the data from the webpage
import datetime

In [2]:
# Obtain the file to scrape soccer matches 
file_name='data/junior.html'
with open(file_name, encoding="utf8") as f:
    stock_soup = BeautifulSoup(f, 'html.parser')


In [13]:
# Defining the match index
m = 0

# Define the Dataframe to store the games
df_games = pd.DataFrame(columns=["date", "home_team","away_team","home_result","away_result"])

# Obtain all matches data in the html
for games in stock_soup.find_all('div', class_=re.compile('event__match event__match--static event__match--oneLine|event__match event__match--static event__match--last event__match--oneLine')):
    for match in games:
        # Obtain date
        if match.attrs['class'][0] == 'event__time':
            d = datetime.datetime.now()
            if len(match.text) == 12:
                d = datetime.date(2019,int(match.text[3:5]),int(match.text[:2]))
            elif len(match.text) == 10:
                d = datetime.date(int(match.text[6:10]),int(match.text[3:5]),int(match.text[:2]))
            df_games.at[m,"date"]=d
        # Obtain scores
        elif match.attrs['class'][0] == 'event__scores':
            home_result = match.text.split('-')[0].strip()
            away_result = match.text.split('-')[1].strip()
            df_games.at[m,"home_result"] = int(home_result[0])
            df_games.at[m,"away_result"] = int(away_result[0])
        # Obtain home team
        elif len(match.attrs['class']) > 1 and match.attrs['class'][1] == 'event__participant--home':
            df_games.at[m,"home_team"] = match.text.upper()
        # Obtain away team
        elif len(match.attrs['class']) > 1 and match.attrs['class'][1] == 'event__participant--away':
            df_games.at[m,"away_team"] = match.text.upper()
    m +=1

# Transform to datetime
df_games['date'] = pd.to_datetime(df_games['date']).dt.date

# New columns to set if junior won
df_games['junior_won'] = df_games.apply(lambda x: 'WON' if ( x['home_team'] == 'JUNIOR' and (x['home_result'] > x['away_result'])) else\
                                                        ( 'WON' if (x['away_team'] == 'JUNIOR' and (x['away_result'] > x['home_result'])) else\
                                                        ( 'LOST' if (x['home_team'] == 'JUNIOR' and (x['home_result'] < x['away_result'])) else\
                                                        ( 'LOST' if (x['away_team'] == 'JUNIOR' and (x['away_result'] < x['home_result'])) else 'DRAW' ))), axis=1)

# Define the critical and medium relevance teams
critical_teams = ['PALMEIRAS (BRA)','FLAMENGO (BRA)','ATL. NACIONAL','AMÉRICA DE CALI','U. MAGDALENA','MILLONARIOS','ALIANZA LIMA (PER)','ATHLETICO-PR (BRA)','BOCA JRS. (ARG)','OLIMPIA (PAR)']
medium_teams = ['ONCE CALDAS', 'DEPORTIVO CALI','DEPORTIVO CALI (COL)','IND. SANTA FE','IND. SANTA FE (COL)' , 'CHAPECOENSE (BRA)', 'SAN LORENZO (ARG)']

# Define the relevance for each match based on the teams
df_games['relevance'] =df_games.apply(lambda x: 'HIGH' if (x['home_team'] in critical_teams or x['away_team'] in critical_teams) else\
                                                          ('MEDIUM' if (x['home_team'] in medium_teams or x['away_team'] in medium_teams) else\
                                                          'LOW'), axis=1)
# Export the dataframe to a CSV
df_games.to_csv('Data\junior.csv')

In [12]:
df_games

,date,home_team,away_team,home_result,away_result,junior_won,relevance
0,2019-10-09,JUNIOR,DEPORTIVO CALI,3,0,WON,MEDIUM
1,2019-10-06,ATL. NACIONAL,JUNIOR,0,1,WON,HIGH
2,2019-10-03,JUNIOR,LA EQUIDAD,1,0,WON,LOW
3,2019-09-30,U. MAGDALENA,JUNIOR,1,1,DRAW,HIGH
4,2019-09-22,JUNIOR,JAGUARES DE CÓRDOBA,3,0,WON,LOW
...,...,...,...,...,...,...,...
355,2014-07-06,JUNIOR,BARRANQUILLA,2,0,WON,LOW
356,2014-07-02,JUNIOR,VALLEDUPAR,3,2,WON,LOW
357,2019-10-25,ATL. NACIONAL,JUNIOR,3,1,LOST,HIGH
358,2014-05-18,JUNIOR,ATL. NACIONAL,1,0,WON,HIGH
